In [1]:
!pip install langchain
!pip install sentence-transformer
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf

# Retrival Chain Pipeline

## Pipeline mainly consist of:
1. Having updated information from sources
2. Splitting the information to reduce number of words to model
3. Converting splitted information to embedding 
4. Adding to any vector database like FAISS (or Weaviate)
5. Perform similiarty search of query in database
6. Retrieving selected document
7. Finally creating a prompt using selected document and query (Prompt Engineering) and getting the output.

In [5]:
!pip install ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 52.2 MB/s eta 0:00:0000:0100:01


In [9]:
!pip install faiss-cpu
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 3.6 MB/s eta 0:00:00 0:00:01


In [10]:
from langchain.chains import RetrievalQAWithSourcesChain
import torch
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
import pickle
import faiss
from sentence_transformers import SentenceTransformer

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [11]:
# Getting VectorEmbedding of Splitted Documents
with open('/kaggle/input/vector-embedding/vectorEmb.pkl', 'rb') as f:
    vector_emb = pickle.load(f)
vector_emb.shape

(104, 768)

In [12]:
index = faiss.IndexFlatL2(vector_emb.shape[1])
index.add(vector_emb) # Adding embedding in FAISS

In [13]:
encoder = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
query = "What is the price of H100?"

query_vector = encoder.encode(query)
query_vector.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(768,)

In [16]:
import numpy as np
ind = index.search(np.array(query_vector).reshape(1,-1), k = 2)

ind[1][0] # Indexes of Similiar document

array([79, 76])

In [18]:
# Getting Documents in text form
with open('/kaggle/input/vector-embedding/document.pkl','rb') as f:
    docs = pickle.load(f)
len(docs)

104

In [27]:
docs

['English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nFREE Credit Score₹100 Cash Reward\n\nFixed Deposits\n\nMy Alerts',
 'My Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nFREE Credit Score₹100 Cash Reward\n\nFixed Deposits\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nChat with Us\n\nDownload App',
 'Follow us on:\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_BUSINESS_AS/MC_ENG_ROS_NWS_BUS_AS_ATF_728\n\nGo PRO @₹99 PRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy',
 'Companies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nHomeNewsBusinessTata Motors, Mahindra gain certificates for production-linked payouts\n\nTrending Topics',
 'Business News LiveAshneer GroverDiffusion Engineers IPO Allotment Status Bank Holidays in OctonberGold Price Today\n\nTata Motors, Mahindra gain certificates fo

In [26]:
selected_docs = [docs[i] for i in ind[1][0]]
selected_docs # Getting Selected documents in Text form

['business at 14.5x EV/EBITDA, and a value of INR 40 per share for stake in Tata Technologies and arrive at a target price of INR 743 per share (from INR 615 earlier), implying an upside of 15.4%.',
 'of 40.8% QoQ.']

In [28]:
whole_text = ''

for st in selected_docs:
    whole_text += " " + st
    
whole_text

' business at 14.5x EV/EBITDA, and a value of INR 40 per share for stake in Tata Technologies and arrive at a target price of INR 743 per share (from INR 615 earlier), implying an upside of 15.4%. of 40.8% QoQ.'

In [30]:
# Giving Prompt to pretrained LLM model 
template = '''
    Read the below paragraph
    {info}
    
    My Question is {question}
'''

query = "What is the price of Tiago iCNG?"

prompt = PromptTemplate(input_variable = ["info", "question"],
                       template = template)

llm = CTransformers(model = "/kaggle/input/llama-3b-ggml/ggml/default/1", 
                    model_type ='llama', 
                    config = {'max_new_tokens': 256,
                             'temperature': 0.01},
                    n_gpu_layers = -1,
#                     device = torch.device("cuda:0")
                   )

response = llm(prompt.format(info = whole_text, question = query))

/tmp/ipykernel_30/220775138.py:21: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt.format(info = whole_text, question = query))


In [31]:
response

'\nComment: I think you should ask this on [Stack Overflow](https://stackoverflow.com/) instead of here.\n\nAnswer: The price of Tata Tiago iCNG is ₹6,49,000 (ex-showroom).'